In [333]:
import json
import subprocess
import papermill as pm
import os
from datetime import datetime
import tempfile
import nbformat
import stat

In [334]:
# Definir el directorio base y temporal
try:
    BASE_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    BASE_DIR = os.getcwd()

TEMP_DIR = os.path.join(BASE_DIR, 'temp')
os.makedirs(TEMP_DIR, exist_ok=True)
os.chmod(TEMP_DIR, 0o777)


In [335]:
# Función para ejecutar notebooks
def ejecutar_notebook(notebook_path, parameters=None):
    if parameters is None:
        parameters = {}
    tmp_output_path = os.path.join(TEMP_DIR, 'output_notebook.ipynb')
    pm.execute_notebook(
        notebook_path,
        tmp_output_path,  # Usar un archivo temporal para el output
        parameters=parameters
    )

In [336]:
COMMON_NOTEBOOK_PATH = os.path.join(BASE_DIR, "01_LecturayAnalisis.ipynb")

In [337]:
# Ejecutar el notebook común
ejecutar_notebook(COMMON_NOTEBOOK_PATH)

Executing:   0%|          | 0/14 [00:00<?, ?cell/s]

In [ ]:
# Variantes de normalización
variantes = ['No_Norm', 'MinMax', 'Std', 'Maxabs', 'Robust']

In [ ]:
# Ejecutar notebooks específicos para cada variante del experimento
for variante in variantes:
    notebook_path = os.path.join(BASE_DIR, f"02_Norm_{variante}.ipynb")
    #output_notebook = os.path.join(OUTPUT_DIR, f"02_Norm_{variante}_{datetime.now().strftime('%Y%m%d')}.ipynb")
    ejecutar_notebook(notebook_path, parameters={})
    
print("Ejecución de notebooks de normalización completada.")

Executing:   0%|          | 0/4 [00:00<?, ?cell/s]

PermissionError: [Errno 13] Permission denied: 'c:\\Users\\Gonzalo\\Documents\\GitHub\\00_tesisaustral\\tesisaustral\\tesisaustral\\tmpz6j9eig_.ipynb'

In [ ]:
for variante in variantes:
    notebook_path = os.path.join(BASE_DIR, f"03_Train_Test.ipynb")  # Usar el nombre original del notebook
    parameters = {
        'variante': variante,
        'test_size': 0.2,
        'random_state': 42  
    }
    ejecutar_notebook(notebook_path, parameters)

print("Ejecución de notebooks de normalización completada.")

Executing:   0%|          | 0/9 [00:00<?, ?cell/s]

Executing:   0%|          | 0/9 [00:00<?, ?cell/s]

Executing:   0%|          | 0/9 [00:00<?, ?cell/s]

Executing:   0%|          | 0/9 [00:00<?, ?cell/s]

Executing:   0%|          | 0/9 [00:00<?, ?cell/s]

Ejecución de notebooks de normalización completada.


In [ ]:
#Parámetros TPOT
tpot_config = {
    "generations": 5,
    "population_size": 20,
    "verbosity": 2
}

In [ ]:
# Guardar la configuración en un archivo JSON para que el script de modelos lo lea
config = {
    "data_paths": {
        "X_train": f"X_train_{variante}.parquet",
        "X_test": f"X_test_{variante}.parquet",
        "y_train": f"y_train_{variante}.parquet",
        "y_test": f"y_test_{variante}.parquet"
    },
    "tpot_config": tpot_config,
    "autosklearn_config": autosklearn_config
}

In [ ]:
config_path = os.path.join(BASE_DIR, 'control_config.json')
with open(config_path, 'w') as f:
    json.dump(config, f)

In [ ]:
# Ejecutar el script de modelos
subprocess.run(['python', os.path.join(BASE_DIR, '04_Auto_Models.py')])